^fill^README.md^here^

- - -

^fill^theory.md^here^

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'surface_energy'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_serial'
mpi_command = None
lib_directory = '../../../library'

#### 2.2 Specify the potenital and elemental symbols

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use (required).

- __symbols__ is a list of the elemental model symbols of potential to associate with the unique atom types of system (required).

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __potential_path__ gives the path to the potential_LAMMPS reference record to use.

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file)



In [4]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 'potential_LAMMPS', potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
with open(potential_path) as f:
    potential = lmp.Potential(f, potential_dir_path)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.3 Specify the prototype unit cell system
 
- __ucell__ (required) is an atomman.System representing a fundamental unit cell of the system .

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __prototype_path__ gives the path to the crystal_prototype reference record to use.

- __box_parameters__ defines the initial guess box parameters to scale ucell and system by.

In [5]:
prototype_name = 'A1--Cu--fcc'
box_parameters = [3.52, 3.52, 3.52]

# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 'crystal_prototype', prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]
print('# of atoms in ucell =', ucell.natoms)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

# of atoms in ucell = 4


#### 2.4 Specify the defect 

__2.4a Specify the defect model__

- __surface_name__ gives the name of the free_surface reference record in the iprPy library to use for potential. 

- __surface_path__ gives the path to the free_surface reference record to use.

- __surface_model__ is a DataModelDict of a free_surface record.

In [6]:
#surface_name = 'A1--Cu--fcc--100'
surface_name = 'A1--Cu--fcc--110'
#surface_name = 'A1--Cu--fcc--111'

# Define surface_path using lib_directory and surface_name
surface_path = os.path.abspath(os.path.join(lib_directory, 'free_surface', surface_name+'.json')) 

# Load free-surface record as a DataModelDict
with open(surface_path) as f:
    surface_model = DM(f)

print('Successfully loaded defect record for', surface_model['free-surface']['id'])

Successfully loaded defect record for A1--Cu--fcc--110


__2.4b Extract the free surface defect parameters__

- __surface_kwargs__ is a dictionary containing parameters for generating the defect. Values are extracted from a free-surface record and uniquely define a type of free surface. Alternatively, the associated terms can be directly defined in the next sections. Included keywords are:

    - __crystallographic-axes__ specifies how to orient the system. Subelements define each of the x-, y-, and z-axes.
    
    - __cutboxvector__ specifies which box vector to apply the free surface cut to.
    
    - __atomshift__ specifies a rigid body shift of all atoms in the initial system.

In [7]:
# Extract defect parameters    
surface_kwargs = surface_model['free-surface']['calculation-parameter']
        
print(surface_kwargs.json(indent=4))

{
    "x_axis": " 0  0  1", 
    "y_axis": " 1 -1  0", 
    "z_axis": " 1  1  0", 
    "cutboxvector": "c", 
    "atomshift": "0.0 0.0 0.1"
}


#### 2.5 Generate the initial system

- __system__ is an atomman.System to perform the scan on (required).

- __x_axis__ is the 3D crystal vector of ucell to align with the x-axis of system.

- __y_axis__ is the 3D crystal vector of ucell to align with the y-axis of system.

- __z_axis__ is the 3D crystal vector of ucell to align with the z-axis of system.

- __atomshift__ applies a rigid-body shift to all atoms in the system.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

In [8]:
sizemults = [5,5,5]

# -------------- Defect parameters --------------- #
x_axis = np.array(surface_kwargs['x_axis'].split(), dtype=float)
y_axis = np.array(surface_kwargs['y_axis'].split(), dtype=float)
z_axis = np.array(surface_kwargs['z_axis'].split(), dtype=float)
atomshift = np.array(surface_kwargs['atomshift'].split(), dtype=float)

# -------------- Derived parameters -------------- #
# Copy ucell to initialsystem
system = deepcopy(ucell)

# Build axes from x_axis, y_axis and z_axis
axes = np.array([x_axis, y_axis, z_axis])

# Rotate using axes_array
system = am.rotate_cubic(system, axes)

# Apply atomshift
shift = (atomshift[0] * system.box.avect 
         + atomshift[1] * system.box.bvect 
         + atomshift[2] * system.box.cvect)
pos = system.atoms_prop(key='pos')
system.atoms_prop(key='pos', value=pos+shift)

# Apply sizemults
system.supersize(*sizemults)

print('# of atoms in system =', system.natoms)

# of atoms in system = 1000


#### 2.5 Specify calculation-specific run parameters

- __cutboxvector__ defines which of the three system box vector boundaries ($a$, $b$, or $c$) the system will be cut along to create the free surface.

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

In [9]:
cutboxvector = surface_kwargs['cutboxvector']
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 min.template

In [10]:
with open('min.template', 'w') as f:
    f.write("""^fill^min.template^here^""")

#### 3.2 surface_energy()

In [11]:
^fill^calc_surface_energy.py(surface_energy)^here^

#### 3.3 relax_system()

In [12]:
^fill^calc_surface_energy.py(relax_system)^here^

### 4. Run calculation function(s)

In [13]:
results_dict = surface_energy(lammps_command, 
                              system, 
                              potential,
                              symbols,  
                              mpi_command = mpi_command,
                              cutboxvector = cutboxvector,
                              etol = energytolerance,
                              ftol = forcetolerance,
                              maxiter = maxiterations,
                              maxeval = maxevaluations,
                              dmax = maxatommotion)

In [14]:
results_dict.keys()

['dumpfile_surf',
 'E_surf_f',
 'E_total_surf',
 'E_coh',
 'A_surf',
 'E_total_base',
 'dumpfile_base']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display results in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __e_A_unit__ is the energy per area to report the surface energy in.

In [15]:
length_unit = 'angstrom'
energy_unit = 'eV'

#e_A_unit = energy_unit+'/'+length_unit+'^2'
e_A_unit = 'mJ/m^2'

#### 5.2 Print $E_{coh}$, $A_{surface}$, and $E_{surface}^f$

In [16]:
print('E_coh =  ', uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('A_surface =', uc.get_in_units(results_dict['A_surf'], length_unit+'^2'), length_unit+'^2')
print('E_surface_f =', uc.get_in_units(results_dict['E_surf_f'], e_A_unit), e_A_unit)

E_coh =   -4.44999999835 eV
A_surface = 438.066793081 angstrom^2
E_surface_f = 2049.39393517 mJ/m^2
